In [1]:
pip list

Package                   Version
------------------------- -----------
annotated-types           0.7.0
anthropic                 0.49.0
anyio                     4.9.0
asttokens                 3.0.0
attrs                     25.3.0
beautifulsoup4            4.13.3
bleach                    6.2.0
certifi                   2025.1.31
charset-normalizer        3.4.1
click                     8.1.8
comm                      0.2.2
contourpy                 1.3.1
cycler                    0.12.1
debugpy                   1.8.11
decorator                 5.2.1
defusedxml                0.7.1
distro                    1.9.0
exceptiongroup            1.2.2
executing                 2.1.0
fastapi                   0.115.12
fastjsonschema            2.21.1
filelock                  3.18.0
fonttools                 4.56.0
fsspec                    2025.3.2
greenlet                  3.1.1
groq                      0.22.0
h11                       0.14.0
httpcore                  1.0.7
httpx       

In [3]:
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List
from fastapi import FastAPI

app = FastAPI()

# Define Models
class Fruit(BaseModel):
    name: str

class Fruits(BaseModel):
    fruits: List[Fruit]

# CORS Middleware
origins = [
    "http://localhost:3000",  # Corrected URL format
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # This must be a list
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# In-memory database
memory_db = {"fruits": []}

# Get Fruits
@app.get("/fruits", response_model=Fruits)
def get_fruits():
    return Fruits(fruits=memory_db["fruits"])

# Add a Fruit
@app.post("/fruits", response_model=Fruit)
def add_fruit(fruit: Fruit):
    memory_db["fruits"].append(fruit)
    return fruit

# Run the app
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


RuntimeError: asyncio.run() cannot be called from a running event loop

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

# Initialize the Ollama Model (adjust 'model' name as per your local Ollama setup)
llm = ChatOllama(model="gemma3:1b")  # or any model you've pulled like "mistral", "llama2", etc.

# Define System Prompt
system_prompt = SystemMessage(content="You are a helpful AI Assistant. Answer the User's queries succinctly in one sentence.")

# Start Storage for Historical Message History
messages = [system_prompt]

while True:
    # Get User's Message
    user_input = input("\nUser: ")
    
    if user_input.lower() == "exit":
        break

    user_message = HumanMessage(content=user_input)

    # Extend Messages List With User Message
    messages.append(user_message)

    # Pass Entire Message Sequence to LLM to Generate Response
    response = llm.invoke(messages)
    
    print("\nAI Message:", response.content)

    # Add AI's Response to Message List
    messages.append(response)

# Print all messages stored in the message list
for i, message in enumerate(messages):
    print(f"\nMessage {i+1} - {message.type.upper()}: {message.content}")




AI Message: Hello there! How can I help you today?

AI Message: Okay! Nice to meet you, Yash.

AI Message: I’m a large language model, named Gemma.

AI Message: Your name is Yash!

Message 1 - SYSTEM: You are a helpful AI Assistant. Answer the User's queries succinctly in one sentence.

Message 2 - HUMAN: hi

Message 3 - AI: Hello there! How can I help you today?

Message 4 - HUMAN: my name ios yash

Message 5 - AI: Okay! Nice to meet you, Yash.

Message 6 - HUMAN: whtas your name

Message 7 - AI: I’m a large language model, named Gemma.

Message 8 - HUMAN: whats myu name

Message 9 - AI: Your name is Yash!


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

reflection_prompt_template = """
You are analyzing conversations about research papers to create memories that will help guide future interactions. Your task is to extract key elements that would be most helpful when encountering similar academic discussions in the future.

Review the conversation and create a memory reflection following these rules:

1. For any field where you don't have enough information or the field isn't relevant, use "N/A"
2. Be extremely concise - each string should be one clear, actionable sentence
3. Focus only on information that would be useful for handling similar future conversations
4. Context_tags should be specific enough to match similar situations but general enough to be reusable

Output valid JSON in exactly this format:
{{
    "context_tags": [              // 2-4 keywords that would help identify similar future conversations
        string,                    // Use field-specific terms like "deep_learning", "methodology_question", "results_interpretation"
        ...
    ],
    "conversation_summary": string, // One sentence describing what the conversation accomplished
    "what_worked": string,         // Most effective approach or strategy used in this conversation
    "what_to_avoid": string        // Most important pitfall or ineffective approach to avoid
}}

Examples:
- Good context_tags: ["transformer_architecture", "attention_mechanism", "methodology_comparison"]
- Bad context_tags: ["machine_learning", "paper_discussion", "questions"]

- Good conversation_summary: "Explained how the attention mechanism in the BERT paper differs from traditional transformer architectures"
- Bad conversation_summary: "Discussed a machine learning paper"

- Good what_worked: "Using analogies from matrix multiplication to explain attention score calculations"
- Bad what_worked: "Explained the technical concepts well"

- Good what_to_avoid: "Diving into mathematical formulas before establishing user's familiarity with linear algebra fundamentals"
- Bad what_to_avoid: "Used complicated language"

Additional examples for different research scenarios:

Context tags examples:
- ["experimental_design", "control_groups", "methodology_critique"]
- ["statistical_significance", "p_value_interpretation", "sample_size"]
- ["research_limitations", "future_work", "methodology_gaps"]

Conversation summary examples:
- "Clarified why the paper's cross-validation approach was more robust than traditional hold-out methods"
- "Helped identify potential confounding variables in the study's experimental design"

What worked examples:
- "Breaking down complex statistical concepts using visual analogies and real-world examples"
- "Connecting the paper's methodology to similar approaches in related seminal papers"

What to avoid examples:
- "Assuming familiarity with domain-specific jargon without first checking understanding"
- "Over-focusing on mathematical proofs when the user needed intuitive understanding"

Do not include any text outside the JSON object in your response.

Here is the prior conversation:

{conversation}
"""

reflection_prompt = ChatPromptTemplate.from_template(reflection_prompt_template)

reflect = reflection_prompt | llm | JsonOutputParser()


def format_conversation(messages):
    
    # Create an empty list placeholder
    conversation = []
    
    # Start from index 1 to skip the first system message
    for message in messages[1:]:
        conversation.append(f"{message.type.upper()}: {message.content}")
    
    # Join with newlines
    return "\n".join(conversation)

conversation = format_conversation(messages)

print(conversation)



HUMAN: hi
AI: Hello there! How can I help you today?
HUMAN: my name ios yash
AI: Okay! Nice to meet you, Yash.
HUMAN: whtas your name
AI: I’m a large language model, named Gemma.
HUMAN: whats myu name
AI: Your name is Yash!


In [4]:
reflection = reflect.invoke({"conversation": conversation})

print(reflection)

{'context_tags': ['name', 'user_greeting', 'user_introduction'], 'conversation_summary': 'The user introduced themselves as Yash, and the AI confirmed their name is Yash.', 'what_worked': "Leveraging the user's name as a starting point for context establishment and confirming identity.", 'what_to_avoid': 'Assuming a pre-existing knowledge of AI models or language models without explicitly stating it.'}


In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document

# Set up embeddings using Ollama (assuming Ollama is running locally)
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")  # or any embedding model pulled via Ollama

# Example documents
docs = [
    Document(page_content="Understanding attention in transformers."),
    Document(page_content="Control groups are important in experimental setups."),
]

# Create FAISS store
vectorstore = FAISS.from_documents(docs, embeddings)

# Save for later
vectorstore.save_local("faiss_ollama_store")


/tmp/ipykernel_25522/3863004761.py:6: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")  # or any embedding model pulled via Ollama


In [24]:
# Reload vectorstore
vs = FAISS.load_local(
    "/home/yash/Desktop/Langchain/backend/faiss_ollama_store",
    embeddings,
    allow_dangerous_deserialization=True
)


# Run similarity search
query = "How does attention mechanism work?"
results = vs.similarity_search(query)

for r in results:
    print("Match:", r.page_content)


Match: Understanding attention in transformers.
Match: Control groups are important in experimental setups.
